**Welcome to the Kaggle 5-day Agents course!**

This notebook is your first step into building AI agents. An agent can do more than just respond to a prompt — it can **take actions** to find information or get things done.

In this notebook, you'll:

- ✅ Install [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Configure your API key to use the Gemini model
- ✅ Build your first simple agent
- ✅ Run your agent and watch it use a tool (like Google Search) to answer a question


## ⚙️ Section 1: Setup

In [1]:
#create venv
# python -m venv venv
#activate venv
# .\venv\Scripts\activate
#install dependencies
# pip install -r requirements.txt

In [ ]:
# authentication setup
import os
from dotenv import load_dotenv

# laod .env file from root directory which has the API key
load_dotenv('../../.env')

try:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if GOOGLE_API_KEY is None:
        raise ValueError("GOOGLE_API_KEY not found in environment variables.")
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("GOOGLE_API_KEY successfully set.")
except Exception as e:
   print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}"
    )

GOOGLE_API_KEY successfully set.


In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## 🤖 Section 2: Your first AI Agent with ADK
### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

### 2.2 Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [5]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [6]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner initialized.")

✅ Runner initialized.


👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK](https://google.github.io/adk-docs/runtime/).

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

In [7]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
helpful_assistant > The Google Agent Development Kit (ADK) is a flexible, modular, and open-source framework designed to simplify the creation, deployment, and orchestration of AI agents and multi-agent systems. It aims to make agent development more akin to traditional software development, allowing developers to build complex agentic architectures that can range from simple tasks to intricate workflows. The ADK is optimized for the Google ecosystem, particularly with Gemini models and Vertex AI, but it is model-agnostic and compatible with other frameworks.

The SDK is available in the following languages:
*   **Python**
*   **Java**
*   **Go**

More languages are expected to be supported in the future.


In [9]:
response = await runner.run_debug(
    "Who won the FIFA World Cup 2022? Provide a brief summary of the final match."
)


 ### Continue session: debug_session_id

User > Who won the FIFA World Cup 2022? Provide a brief summary of the final match.
helpful_assistant > Argentina won the FIFA World Cup 2022, defeating France in a dramatic final match. The game, played on December 18, 2022, at Lusail Stadium in Qatar, ended in a 3-3 draw after extra time, with Argentina ultimately winning 4-2 in a penalty shootout. This victory marked Argentina's third World Cup title and their first since 1986.

The final match is widely considered one of the greatest World Cup finals in history due to its numerous momentum swings and high scoring. Argentina took a 2-0 lead in the first half with goals from Lionel Messi (penalty) and Ángel Di María. France mounted a stunning comeback in the second half with two quick goals from Kylian Mbappé, including a penalty, to tie the game at 2-2. In extra time, Messi scored again to put Argentina ahead 3-2, but Mbappé responded with another penalty to complete his hat-trick and level 

## 💻 Section 3: Try the ADK Web Interface

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

In [10]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in c:\Users\Shahin\Work Things\Works n practices\projects\google-agents-lab\labs\day01_intro\sample-agent:
- .env
- __init__.py
- agent.py



In [ ]:
# since it's running in a notebook in localhost, we need to set the url_prefix
# url_prefix = get_adk_proxy_url()
# !adk web --url_prefix {url_prefix}

In [ ]:
# to run the ui in localhost, in the this directory(labs/day01_intro) in terminal run:
# adk web